In [2]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup

#### Scrap and save the data

In [3]:
df = pd.DataFrame(columns=["Average Rank", "Film", "Year"])
count = 0

url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'

html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')

tables = data.find_all("tbody")
rows = tables[0].find_all("tr")

for row in rows:
    col = row.find_all('td')
    if len(col) != 0:
        year_value = col[2].contents[0].strip()
        year = int(year_value) if year_value.isdigit() else None

        data_dict = {
            "Average Rank": col[0].contents[0],
            "Film": col[1].contents[0],
            "Year": year
        }
        df1 = pd.DataFrame(data_dict, index=[0])
        df = pd.concat([df, df1], ignore_index=True)
        count+=1

In [4]:
csv_path = "top_films.csv"

df.to_csv(csv_path, index=False)

In [5]:
db_name = 'Movies.db'
table_name = 'Top_films'

conn = sqlite3.connect(db_name)
df.to_sql(table_name, conn, if_exists='replace', index=False)

108

In [37]:
# df[(df.Year>=2000) & (df.Year<=2005)].sort_values(by="Year")

In [6]:
query_statement = "select * from Top_films limit 5"
sql_df = pd.read_sql(query_statement, conn)
sql_df

,Average Rank,Film,Year
0,1,The Godfather,1972
1,2,Citizen Kane,1941
2,3,Casablanca,1942
3,4,"The Godfather, Part II",1974
4,5,Singin' in the Rain,1952


In [7]:
conn.close()